In [10]:
import torch
torch.cuda.is_available()
import numpy as np
import utils
from collections import Counter

'1.11.0'

In [2]:
def make_imb_data(max_num, class_num, gamma, inv=False, bal=False):
    print('max_num={}'.format(max_num))
    mu = np.power(1/gamma, 1/(class_num - 1))
    class_num_list = []
    for i in range(class_num):
        if(inv):
            class_num_list.append(int(max_num * np.power(mu, class_num-i-1)))
        else:
            class_num_list.append(int(max_num * np.power(mu, i)))
    if(bal):
        per_class = sum(class_num_list)/class_num
        class_num_list = [int(per_class) for i in range(class_num)]
    print(class_num_list)
    return list(class_num_list)

N_SAMPLES_PER_CLASS = make_imb_data(1250, 10, 100 ,False, False)
U_SAMPLES_PER_CLASS = make_imb_data(3 * 1250, 10, 100, False, False)
N_SAMPLES_PER_CLASS_T = torch.Tensor(N_SAMPLES_PER_CLASS)

train_labeled_set, train_unlabeled_set, test_set = utils.get_cifar10('/home/shrinivas/Apoorva/Datasets', N_SAMPLES_PER_CLASS
, U_SAMPLES_PER_CLASS)

max_num=1250
[1250, 749, 449, 269, 161, 96, 58, 34, 20, 12]
max_num=3750
[3750, 2248, 1347, 807, 484, 290, 174, 104, 62, 37]
Files already downloaded and verified
#Labeled: 3098 #Unlabeled: 12401


In [3]:
target_dist = torch.nn.functional.normalize(torch.ones([10,10]))
labeled_loader = torch.utils.data.DataLoader(train_labeled_set, batch_size=128, num_workers=8, shuffle=True)

In [4]:
pseudo_labels = [train_unlabeled_set.targets[i] for i in np.arange(len(train_unlabeled_set.targets))]

def get_weights(pseudo_labels, lbl, target_dist):
    """Returns the sampling weights for each data instance in the dataset 

    Args:
        pseudo_labels (list): list of pseudo-labels generated by the model
        lbl (tensor): labels present in one batch obtained from the dataloader
        target_dist (tensor): target distribution obtained by dM/dCij

    Returns:
        list: list of instance-wise weights to be passed into the sampler
    """

    wts = []
    for t in pseudo_labels:
        wts.append(target_dist[lbl][t].numpy())
    return wts

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for imgs, lbls, idxs in labeled_loader:
    imgs = imgs.to(device)
    lbls = lbls.to(device)
    for i in lbls:
        pl_weights = get_weights(pseudo_labels, i, target_dist)
        sampler = torch.utils.data.WeightedRandomSampler(weights= pl_weights, num_samples = 1, replacement = True)
        


ValueError: too many values to unpack (expected 2)

In [8]:
y_train = [train_labeled_set.targets[i] for i in np.arange(len(train_labeled_set.targets))]
b = []
for t in y_train:
    b.append(1)

sample_wts = np.array(b)

sampler = torch.utils.data.WeightedRandomSampler(weights= 1/sample_wts, num_samples = len(b), replacement = True)
print(np.array(list(sampler)).shape)


(3098,)


In [ ]:
class UnlabeledDataLoader(torch.utils.data.DataLoader):

    def __init__(self, target_dist):
        super(UnlabeledDataLoader, self).__init__()
        self.target_dist = target_dist

    def get_batch(self, labels):
        for i in labels:
            # define logic for weights for each sampler 